## Training Model on Train Images
**Data Preparation:**<br>
- Use ImageDataGenerator for real-time data augmentation and preprocessing.
- Split the dataset into training and validation subsets.

**Model Configuration:**<br>
- Load a pre-trained ResNet-50 model as the base.
- Add a custom head (layers) to the base model for the specific classification task.
- Freeze the layers of the base model to retain their pre-trained weights.

**Handling Class Imbalance:**<br>
- Calculate class weights to give more importance to under-represented classes during training.

**Model Compilation:**<br>
- Use the Adam optimizer.
- Set the loss function as categorical_crossentropy suitable for multi-class classification.
- Track AUC, Precision, and Recall as metrics.

**Training Callbacks:**<br>
- arlyStopping: Stop training early if validation loss doesn't improve for a set number of epochs.
- ModelCheckpoint: Save the best model weights based on validation loss.
- ReduceLROnPlateau: Reduce learning rate when validation loss plateaus.
- TensorBoard: Enable visualization of training metrics and model profiling.

**Model Training:**<br>
- Train the model for a set number of epochs or until early stopping criteria are met.
- Use the generated data from the ImageDataGenerator.
- Apply class weights to handle class imbalance during training.

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from sklearn.utils import compute_class_weight

## Define & Create Data Generators

In [2]:
# Data generator for training with data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # preprocess for ResNet-50
    rotation_range=20,  # Augmentation
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# Data generator for validation without data augmentation
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Data generator for testing without data augmentation
# Note: This is technically the same as val_datagen, but for clarity, defined it separately.
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    'D:/cancer/organized/train',
    target_size=(224, 224),  # just for clarity and safety, resizing is already done
    batch_size=32,
    class_mode='categorical'  # labels are strings, will be one-hot-encoded
)

# Validation data generator
validation_generator = val_datagen.flow_from_directory(
    'D:/cancer/organized/validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Testing data generator
test_generator = test_datagen.flow_from_directory(
    'D:/cancer/organized/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)



Found 6009 images belonging to 7 classes.
Found 2003 images belonging to 7 classes.
Found 2003 images belonging to 7 classes.


## Load ResNet-50

In [3]:
# exclude output layer, initialized with imagenet weights instead of random to converge faster
base_model = ResNet50(weights='imagenet', include_top=False)

## Add custom layer

In [4]:
# Get the output tensor of the base model (ResNet-50 in your case). 
# This will be the starting point for our custom head.
x = base_model.output

# Add a Global Average Pooling (GAP) layer. This layer will average the spatial dimensions 
# (height and width) of the input tensor, resulting in a tensor of shape (batch_size, channels).
# It's a way to reduce the spatial dimensions while keeping the depth (channels).
x = GlobalAveragePooling2D()(x)

# Add a Dense (fully connected) layer with 2048 neurons and ReLU activation.
# This will learn to make high-level decisions based on the features extracted by the previous layers.
x = Dense(2048, activation='relu')(x)

# Add a Batch Normalization layer. This layer will normalize the activations of the previous layer 
# (the Dense layer) to have a mean of 0 and a standard deviation of 1. 
# This can help in stabilizing and speeding up the training process.
x = BatchNormalization()(x)

# Add the final Dense layer with as many neurons as there are classes in the dataset.
# The softmax activation function ensures the output values are in the range [0, 1] and sum to 1, 
# making them interpretable as class probabilities.
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model using the base model's input and our custom head's output.
# This connects the base model and the custom head into a single model that we can train.
model = Model(inputs=base_model.input, outputs=predictions)

## Freeze Base Model Layers

In [5]:
#Freeze layers of the ResNet-50 model to use it as feature extractor
for layer in base_model.layers:
    layer.trainable = False

## Calculate Class Weights

In [6]:
# remember:passing the arguments positionally did not work (wrong order), had to use keyword args
# Extract unique classes and corresponding labels
classes = np.unique(train_generator.classes)
y_integers = train_generator.labels

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', 
                                     classes=classes, 
                                     y=y_integers)

# Convert the class weights to a dictionary format
class_weight_dict = dict(zip(classes, class_weights))
class_weight_dict

{0: 4.379737609329446,
 1: 2.787105751391466,
 2: 1.3006493506493506,
 3: 12.440993788819876,
 4: 1.2850727117194183,
 5: 0.21338020666879728,
 6: 10.099159663865546}

## Metrics for imbalanced classes

In [7]:
metrics = [
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

## Compile Model

In [8]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=metrics)

## TensorBoard Callback

In [9]:
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

##  Callbacks

In [10]:
# Define the directory name
checkpoint_dir = 'checkpoints'

# Create the directory
os.makedirs(checkpoint_dir, exist_ok=True)

In [11]:
early_stopping = EarlyStopping(
    monitor='val_auc', 
    patience=10, 
    verbose=1, 
    restore_best_weights=True
)

checkpoint_path = os.path.join(checkpoint_dir, 'best_weights.h5')
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_auc',
    mode='max',
    verbose=1,
    save_format='h5'  # explicitly set to 'h5' format
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_auc', 
    factor=0.2, 
    patience=5, 
    min_lr=1e-6, 
    verbose=1
)

## Train

In [12]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    class_weight=class_weight_dict,
    callbacks=[early_stopping, checkpoint, reduce_lr, tensorboard_callback]
)

Epoch 1/100
187/187 [==============================] - ETA: 0s - loss: 2.1790 - auc: 0.7777 - precision: 0.4742 - recall: 0.4076
Epoch 1: val_auc improved from -inf to 0.91347, saving model to checkpoints\best_weights.h5


C:\Users\JasmindelSolar\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


187/187 [==============================] - 298s 2s/step - loss: 2.1790 - auc: 0.7777 - precision: 0.4742 - recall: 0.4076 - val_loss: 1.1813 - val_auc: 0.9135 - val_precision: 0.6983 - val_recall: 0.6336 - lr: 0.0010
Epoch 2/100
187/187 [==============================] - ETA: 0s - loss: 1.1976 - auc: 0.8878 - precision: 0.6452 - recall: 0.5290
Epoch 2: val_auc did not improve from 0.91347
187/187 [==============================] - 302s 2s/step - loss: 1.1976 - auc: 0.8878 - precision: 0.6452 - recall: 0.5290 - val_loss: 3.6267 - val_auc: 0.6700 - val_precision: 0.2855 - val_recall: 0.2525 - lr: 0.0010
Epoch 3/100
187/187 [==============================] - ETA: 0s - loss: 1.0484 - auc: 0.9066 - precision: 0.6775 - recall: 0.5606
Epoch 3: val_auc improved from 0.91347 to 0.91983, saving model to checkpoints\best_weights.h5
187/187 [==============================] - 301s 2s/step - loss: 1.0484 - auc: 0.9066 - precision: 0.6775 - recall: 0.5606 - val_loss: 1.0554 - val_auc: 0.9198 - val_pr

Epoch 21/100
187/187 [==============================] - ETA: 0s - loss: 0.4951 - auc: 0.9666 - precision: 0.8190 - recall: 0.7152
Epoch 21: val_auc did not improve from 0.95396
187/187 [==============================] - 289s 2s/step - loss: 0.4951 - auc: 0.9666 - precision: 0.8190 - recall: 0.7152 - val_loss: 0.7814 - val_auc: 0.9515 - val_precision: 0.7648 - val_recall: 0.6769 - lr: 8.0000e-06
Epoch 22/100
187/187 [==============================] - ETA: 0s - loss: 0.5033 - auc: 0.9661 - precision: 0.8148 - recall: 0.7060Restoring model weights from the end of the best epoch: 12.

Epoch 22: val_auc did not improve from 0.95396

Epoch 22: ReduceLROnPlateau reducing learning rate to 1.6000001778593287e-06.
187/187 [==============================] - 289s 2s/step - loss: 0.5033 - auc: 0.9661 - precision: 0.8148 - recall: 0.7060 - val_loss: 0.7891 - val_auc: 0.9507 - val_precision: 0.7618 - val_recall: 0.6739 - lr: 8.0000e-06
Epoch 22: early stopping


In [15]:
history.history.keys()

dict_keys(['loss', 'auc', 'precision', 'recall', 'val_loss', 'val_auc', 'val_precision', 'val_recall', 'lr'])